<div class="alert alert-block alert-info">

##  Praxisbeispiel: Grayscale-Filter (Graustufen-Filter)

#### Ein Grayscale-Filter (Graustufenfilter) ist ein Werkzeug, das in der Bildbearbeitung verwendet wird, um ein farbiges Bild in ein Graustufenbild zu konvertieren. $\newline$
<div>
    <img src="kingfisher-grayscale-filter.png" style="width:50%;"/>
    </div>
    
#### <b>Anwendungen:</b> Textur- und Strukturanalyse, Datenreduktion, Ästhetik, etc.

#### <b>Einfachste Umsetzung:</b> Die RGB-Kanäle werden für jedes Pixel durch das arithmetische Mittel ersetzt $R=G=B:=\frac{R+G+B}{3}$.
</div>

# 1. Pure Python

In [ ]:
import time #for profiling
from PIL import Image #PIL = Python Imaging Library

In [ ]:
def grayscale_filter_py(pixel_size, bytes):
    """Applies grayscale filter to bytes of an RGB image"""
    
    for i in range(0,3*pixel_size,3):   
        r,g,b = bytes[i], bytes[i+1], bytes[i+2]
        avg = (r+g+b)//3
        bytes[i] = bytes[i+1] = bytes[i+2] = avg
    
    return None

<div class="alert alert-block alert-success">

### Aufgabe 1: 
#### Warum wird der Operator // zur Division verwendet?
</div>

### Testbild

In [ ]:
test_image = Image.open('C:/Users/LENOVO/Pictures/Hochschule_Trier/trier.jpg') #test image from Pixabay
print(test_image.mode, test_image.size)

In [ ]:
def show_image(image):
    """Auxiliary function: Displays a smaller picture without altering the original"""
    thumbnail = image.resize((test_image.width//5, test_image.height//5))
    print(image.mode, image.size)
    display(thumbnail)

In [ ]:
show_image(test_image)

### Anwendung und Profiling

In [ ]:
size = test_image.width*test_image.height
img_bytes = bytearray(test_image.tobytes())

t0 = time.time()
grayscale_filter_py(size, img_bytes)
t1 = time.time()

delta_t_py = t1-t0
print("Execution time (Python): ", round(delta_t_py,3), " seconds.")

In [ ]:
result = Image.frombytes(test_image.mode, test_image.size, bytes(img_bytes)) 
show_image(result)

<div class="alert alert-block alert-success">

### Aufgabe 2: 
#### Stellen Sie eine grobe Überschlagsrechnung an: Wie schnell sollte ein Rechner mit 4 GFLOPS, den Algorithmus für unser Testbild ausführen können?

</div>

# 2. C-Bibliothek

<div class="alert alert-block alert-success">
    
### Aufgabe 3: 
#### Welchen Datentyp verwenden wir in C um ein Byte darzustellen, welches einen Farbwert repräsentieren soll?
</div>

In [ ]:
import ctypes

In [ ]:
c_lib =ctypes.CDLL("./libgrayscale_c.dll")

In [ ]:
size = test_image.width*test_image.height
img_bytes = test_image.tobytes()

t0 = time.time()
c_lib.grayscale_filter(size, img_bytes)
t1 = time.time()

delta_t_C = t1-t0
print("Execution time (C Lib.):", round(delta_t_C,3), " seconds.")

In [ ]:
print("Factor: "+ str(round(delta_t_py/delta_t_C,1)))

In [ ]:
result = Image.frombytes(test_image.mode, test_image.size, img_bytes) 
show_image(result)

# 3. C-Extension als eigenständiges Modul

In [ ]:
import grayscale_module as gs

In [ ]:
help(gs)

In [ ]:
size = test_image.width*test_image.height
img_bytes = test_image.tobytes()

t0 = time.time()
gs.grayscale(size, img_bytes)
t1 = time.time()

delta_t_C_module = t1-t0
print("Execution time (C Module):", round(delta_t_C_module,3), " seconds.")

In [ ]:
print("Factor: "+ str(round(delta_t_py/delta_t_C_module,1)))

In [ ]:
result = Image.frombytes(test_image.mode, test_image.size, img_bytes)
show_image(result)

# 4. Results

In [ ]:
from tabulate import tabulate

In [ ]:
head = ["Execution Time [sec]", "Python Execution Time / Execution Time"]

In [ ]:
data = [
    ["Pure Python", round(delta_t_py,3), 1.0],
    ["C Library (ctypes)", round(delta_t_C,3), round(delta_t_py/delta_t_C,1)],
    ["C Module (C-Extension)", round(delta_t_C_module,3), round(delta_t_py/delta_t_C_module,1)]]

In [ ]:
print(tabulate(data, headers=head, tablefmt="fancy_grid"))

## Wie schnell ist eigentlich der Grayscale-Filter von PIL?

In [ ]:
from PIL import ImageOps

In [ ]:
t0 = time.time()
result = ImageOps.grayscale(test_image)
t1 = time.time()

delta_t_PIL = t1-t0
print("Execution time (PIL):", round(delta_t_PIL,3), " seconds.")

In [ ]:
show_image(result)

#### Appendix

Man könnte den Grayscale-Filter in Python auch auf Pixel-Ebene implementieren, ohne mit einzelnen Bytes zu arbeiten. Dafür bietet PIL geeignete Methoden an (getpixel und putpixel). Dieser Ansatz ist aber extrem langsam, was vor allem an der ineffizienten Methode putpixel liegt. Probieren Sie es aus, mit dem folgenden Code:

In [ ]:
def grayscale_filter_pixelwise(image):
    """Applies grayscale filter to image and returns gray image"""
    
    width, height = image.size
    result = Image.new(image.mode, image.size) 
    
    for x in range(width):
        for y in range(height):            
            r, g, b = image.getpixel((x,y))
            avg = (r+g+b)//3 
            result.putpixel((x,y), (avg,avg,avg)) # Extremely slow function!

    return result

In [ ]:
t0 = time.time()
result = grayscale_filter_pixelwise(test_image)
t1 = time.time()
delta_t_pixels = t1-t0
print("Execution time (Python, pixelwise): ", round(delta_t_pixels,3), " seconds.")

_________

author: Maik Weber, date: March 09, 2024